<a href="https://colab.research.google.com/github/werntzp/hockey-stats/blob/master/common_opponents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Common Opponent Lister - for any team (myhockeyrankings team id needed), go through all their opponents
# looking for common opponents. Print out a list of common opponents and results. 


In [1]:
# install pre-reqs
!pip install pycurl
!pip install beautifulsoup4

# imports 
import pycurl
import pandas as pd 
from bs4 import BeautifulSoup
from io import BytesIO
from google.colab import files


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 233 kB 8.0 MB/s 
  Created wheel for pycurl: filename=pycurl-7.45.1-cp38-cp38-linux_x86_64.whl size=310755 sha256=dfb7e506269a8938b729749ff2050f996cb598fc2a66965dedb0c47ec821206f
  Stored in directory: /root/.cache/pip/wheels/29/9b/97/525e19354d667ab6a101620502bd1df957057f66bee218305d
Successfully built pycurl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# function to build dataframe with all games
def get_games(id, yr):

  # grab the page for team 
  buffer = BytesIO()
  c = pycurl.Curl()
  c.setopt(c.URL, "https://myhockeyrankings.com/team_info.php?y=" + str(yr) + "&t=" + str(id))
  c.setopt(c.WRITEDATA, buffer)
  c.perform()
  c.close()

  # decode into a string we can work with 
  body = buffer.getvalue()
  html = body.decode('iso-8859-1')

  # pass that into beautiful soup 
  soup = BeautifulSoup(html)
  # print(soup)

  # extract the table will all the results 
  table = soup.find(id="team-scores")

  # loop through each row and add to dataframe
  output_rows = []
  for table_row in table.findAll('tr'):
      columns = table_row.findAll('td')
      output_row = []
      for column in columns:
          output_row.append(column.text)
      output_rows.append(output_row)
      #print(output_row)

  # also, grab the name of the team 
  # team_name = soup.find(class_="sitestyle6").text # 2021
  team_name = soup.find(class_="profile-title").text # 2022
  #team_name = element.get_text()

  return output_rows, team_name


In [20]:
# get the wlc09 game list 
our_team_id = "1113"
rows, team_name = get_games(our_team_id, "2022")
df_us = pd.DataFrame(rows, columns = ['Date', 'Time', 'Opponent','Location','Result','NA'])
# print(df_us)

In [27]:
# get the opponent
opponent_id = "3021" # 
rows, team_name = get_games(opponent_id, "2022")
df_opponent = pd.DataFrame(rows, columns = ['Date', 'Time', 'Opponent','Location','Result','NA'])
# print(df_opponent)

In [30]:
# shorten down to unique opponents
opp_list  = df_us.Opponent.unique()
# print(opp_list)

df_common = df_opponent[df_opponent.Opponent.isin(opp_list)]
# print(df_common)

# shut off warning
pd.options.mode.chained_assignment = None

# clean up the dataframe by dropping some colums
del df_common["Time"] # don't care 
del df_common["Location"] # don't care 
del df_common["NA"] # don't care
df_common.drop(df_common.head(1).index, inplace=True) # first row always says none for some reason

# remove any spaces in the team name
team_name = team_name.replace(" ", "")
# avoid any chars that'll impact file system
team_name = team_name.replace("/", "")

# save as csv
df_common.to_csv("/content/" + team_name + "_common_opponents.csv", index=False)

# and display it
df_common

,Date,Opponent,Result
4,2022-09-10,New Hampshire Avalanche (Elite) 09 AAA,3 - 0
7,2022-09-18,Philadelphia Little Flyers 09 AAA,1 - 6
8,2022-09-24,New Jersey Jr Titans 09 AAA,3 - 3
10,2022-10-01,North Jersey Avalanche 09 AAA,5 - 3
11,2022-10-02,Philadelphia Little Flyers 09 AAA,3 - 2
13,2022-10-22,New Jersey Colonials 09 AAA,3 - 6
14,2022-10-23,North Jersey Avalanche 09 AAA,4 - 3
15,2022-10-28,MassConn United (Elite) 09 AAA,0 - 4
19,2022-10-30,Westchester Express (Wegs) 09 AAA,1 - 5
20,2022-11-05,New Jersey Colonials 09 AAA,2 - 5
